# Word generation and prediction using Hidden Markov Model

The aim of this project is to design a algorithm similar to hidden markov model to learn correlations and distributions to
  1. Generate new text from given text corpus dataset
  2. Perform text prediction from given sequence of words
 
Dataset Used: <a src="https://www.kaggle.com/kingburrito666/shakespeare-plays?select=Shakespeare_data.csv">Shakespeare-plays

## **Process**:

1.  Dataset is loaded into pandas and cleaned giving list of all player-lines
2.   Markov model is build and trained
    *   First of all, each line is tokenized from player-lines of dataset
    *  Markov chains are built based on generated tokens
3. ***generate_text()*** function makes use of ***sample_word()*** which given a word  generates a full play sentence  based on pre-built markov chains built using ***build_and_train_markov_model()***
4. ***text_prediction()*** function predicts text based on given sentence 






## Importing Libraries

In [33]:
import string 
import numpy as np
import pandas as pd

## Dataset Manipulation

The following main goals are achieved in this section:


1. Dataset is loaded
2. Uninteresting lines are deleted from the dataset 
3. Lines are converted into a list of string for further processing




In [34]:
df = pd.read_csv ('/content/Shakespeare_data.csv') ##replace content with data when executing 
df.head ()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [35]:
# Keep valid lines (i.e., spoken by a player) only
df = df.dropna (subset = ['Player', 'ActSceneLine'])
df.sample (5)

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
106306,106307,Two Gentlemen of Verona,53.0,2.4.92,VALENTINE,"To see such lovers, Thurio, as yourself:"
4597,4598,Henry VI Part 1,35.0,3.1.154,GLOUCESTER,"So help me God, as I dissemble not!"
76201,76202,Pericles,3.0,1.4.18,CLEON,"I'll then discourse our woes, felt several years,"
24117,24118,A Comedy of Errors,36.0,5.1.104,ADRIANA,"Diet his sickness, for it is my office,"
76348,76349,Pericles,1.0,2.1.5,PERICLES,"Alas, the sea hath cast me on the rocks,"


In [37]:
lines = df ['PlayerLine'].tolist ()
print(lines [10:15])

['All of one nature, of one substance bred,', 'Did lately meet in the intestine shock', 'And furious close of civil butchery', 'Shall now, in mutual well-beseeming ranks,', 'March all one way and be no more opposed']


In [ ]:
#data = "/content/alllines.txt" ##replace content with data when executing 

## Pre-processing data

Let's remove some special characters in each line of text for better results 

In [38]:
def removeSpecial(line):
  return line.translate(str.maketrans("","",string.punctuation))

Creating a hashmap/dictionary for storing key-value pairs


In [39]:
def create_dict(dictionary, key, value):
  if key not in dictionary:
      dictionary[key]=[]
  dictionary[key].append(value)

Creating a probability dict


In [40]:
def create_probability_dict(text_data):
    prob_dict = {}
    text_data_len = len(text_data)
    for item in text_data:
        prob_dict[item] = prob_dict.get(item, 0) + 1
    for key, value in prob_dict.items():
        prob_dict[key] = value / text_data_len
    return prob_dict

Now we need some data-structure to hold initial states and trnasition states

In [41]:
initial_word= {}
second_word = {}
transitions = {}

## Building and Training the Markov Model

One important property about markov model is that the **next step** depends on only the **current step** and not on the past historical steps

For this project, I'm gonna make use of the same 

The training of the Markov model can be divided into the following stages -
1. Cleaning  up data and Tokenisation
2. Building the state pairs(previous and current)
3. Determining the probability distribution

In [42]:
# Trains a Markov model based on the data in data_file
def build_and_train_markov_model():
    for line in lines:

        #tokenizing data
        tokens = removeSpecial(line.rstrip().lower()).split()
        tokens_length = len(tokens)

        #next and current state-pairs
        for i in range(tokens_length):
            token = tokens[i]

            #Initial state need not be calculated for 1st token
            if i == 0:
                initial_word[token] = initial_word.get(token, 0) + 1
            else:
                prev_token = tokens[i - 1]

                ##additional token for last-item
                if i == tokens_length - 1:
                    create_dict(transitions, (prev_token, token), 'END')
                if i == 1:
                    create_dict(second_word, prev_token, token)
                else:
                    prev_prev_token = tokens[i - 2]
                    create_dict(transitions, (prev_prev_token, prev_token), token)
    
    # Normalize the distributions
    initial_word_total = sum(initial_word.values())
    for key, value in initial_word.items():
        initial_word[key] = value / initial_word_total
        
    for prev_word, next_word_list in second_word.items():
        second_word[prev_word] = create_probability_dict(next_word_list)
        
    for word_pair, next_word_list in transitions.items():
        transitions[word_pair] = create_probability_dict(next_word_list)
    
    print('Building and Training finished')

In [43]:
build_and_train_markov_model()

Building and Training finished


## 1.Generating new text from corpus using **Built Hidden Markov Model**

Once we have completed the training, we will have the initial word distribution, second-word distribution and the state transition distributions. Next to generate a text corpus all we need is to write a function to sample out from the above-created distributions.

In [45]:
def sample_word(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

In [44]:
#Fixing our generated text to length 15
number_of_sentences = 12

In [46]:
def generate_text():
    for i in range(number_of_sentences):
        sentence = []
        # Initial word
        word0 = sample_word(initial_word)
        sentence.append(word0)
        # Second word
        word1 = sample_word(second_word[word0])
        sentence.append(word1)
        # Subsequent words untill END
        while True:
            word2 = sample_word(transitions[(word0, word1)])
            if word2 == 'END':
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))

In [47]:
generate_text()

more bound his training such
come if it were otherwise that i might have saved my longing and i more incline to somerset than york
i prithee more
sometime like god bels
words and thereupon i will show thee where they would fly east west north south
dost thou look
no my chuck eros come mine armour
good honest men
not that use them
shallow and another shall
when we were awaked straightway at liberty
truly the souls of friend and will lead you to do


## 2.Performing text prediction given a sequence of words


In [48]:
def text_prediction(text):
        text = removeSpecial(text.lower()).split()
        # Initial word
        word0 = text[0]
        # Second word
        if len(text) == 1:
            word1 = sample_word(second_word[word0])
            text.append(word1)
        else:
            word1 = text[1]
        # Subsequent words untill END
        while True:
            word2 = max(transitions[(word0, word1)], key=transitions[(word0, word1)].get)
            if word2 == 'END':
                break
            text.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(text))

In [49]:
#Testing
text_prediction("walking with")

walking with thee


In [50]:
text_prediction("cudgel him")

cudgel him like a man


## Conclusion

To conclude, i think the text-generation of the markov model is working pretty well and prediction is reasonably good

**References** : 

1. https://towardsdatascience.com/markov-and-hidden-markov-model-3eec42298d75
2. https://en.wikipedia.org/wiki/Hidden_Markov_model
3. https://medium.com/@kangeugine/hidden-markov-model-7681c22f5b9